### Keras Starter 

In [15]:
import bcolz
import numpy as np
import pickle 
from pathlib import Path
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
layers = keras.layers
from keras import metrics
import clr_callback
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
config.log_device_placement=True
sess = tf.Session(config=config)
keras.backend.set_session(sess)

In [16]:
#da = bcolz.open(rootdir='data/processed_main_sorted', mode='r')
#split = int(len(da)*.7)
#split2 = int(len(da)*.8)
split=1000000
split2=split+100000
batch_size = 10000
start=1

In [6]:
model_path = Path('/home/ubuntu/WesternPower-Smartmeter/models/')
model_path.mkdir(exist_ok=True)

In [7]:
#load the column value dictionary
with open("data/cat_map.pkl", "rb") as dictionary_file:
       cat_map = pickle.load(dictionary_file)

In [8]:
class BcolzDataGenerator(keras.utils.Sequence):
    """
    Prepares Data for NN based on Bcolz structure 
       
    """
    def __init__(self, bcolz_directory, batch_size, start_indx, end_indx):
        'Initialization'
        self.da = bcolz.open(rootdir=bcolz_directory, mode='r') 
        self.batch_size = batch_size
        self.start_indx = start_indx
        self.end_indx = end_indx
        self.on_epoch_end()
    def __len__(self):
        'Denotes the number of batches per epoch'                  
        return int(np.floor(self.end_indx  - self.start_indx) / (self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        
        end = self.current_indx+self.batch_size
        X = X=np.nan_to_num(self.da[self.current_indx:end])    
        X = np.delete(X,9,axis=1) # drop the timestamp column   
        Y = X[:,X.shape[1]-1]
        X = X[:,:-1]

        LCLid = X[:,0].astype('int32')
        Acorn_grouped = X[:,2].astype('int32')
        Acorn = X[:,3].astype('int32')
        X = X[:,9:].astype('int32')
        self.current_indx = end
        return [LCLid,Acorn_grouped, Acorn,  X], Y 

    def on_epoch_end(self):
        self.current_indx = self.start_indx
 

In [9]:
TG = BcolzDataGenerator('data/processed_main_sorted', batch_size, 1,split )
VG = BcolzDataGenerator('data/processed_main_sorted', batch_size, split, split2 )

In [10]:
# embedding columns = 'LCLid',  'Acorn_grouped', 'Acorn'
unique_LCLid = len(cat_map['LCLid'])
unique_Acorn_grouped = len(cat_map['Acorn_grouped'])
unique_Acorn = len(cat_map['Acorn'])

In [11]:
LCLid_Input = layers.Input(shape=(1,), dtype='int32', name='LCLid_Input')
Acorn_grouped_Input = layers.Input(shape=(1,), dtype='int32', name='Acorn_grouped_Input')
Acorn_Input = layers.Input(shape=(1,), dtype='int32', name='Acorn_Input')
numeric_input = layers.Input(shape=(40,), dtype='float32', name='Numeric_input')
e1 = layers.Embedding(unique_LCLid, 30,  input_length=1, trainable=True) (LCLid_Input)
e2 = layers.Embedding(unique_Acorn_grouped, 30,  input_length=1, trainable=True) (Acorn_grouped_Input)
e3 = layers.Embedding(unique_Acorn, 30,  input_length=1, trainable=True)(Acorn_Input)
merged_e = layers.concatenate([e1, e2, e3], axis=-1)
merged_e = layers.Flatten()(merged_e)
x= layers.concatenate([merged_e, numeric_input], axis=-1)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dense(64, activation='relu')(x)
main_output = layers.Dense(1, activation='linear', name='main_output')(x)
fcn_model = keras.Model(inputs=[LCLid_Input, Acorn_grouped_Input,Acorn_Input,numeric_input ], outputs=main_output)

In [12]:
fcn_model.compile(optimizer='adam', loss='mse', metrics=[ metrics.mean_squared_error])

In [13]:
fcn_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
LCLid_Input (InputLayer)        (None, 1)            0                                            
__________________________________________________________________________________________________
Acorn_grouped_Input (InputLayer (None, 1)            0                                            
__________________________________________________________________________________________________
Acorn_Input (InputLayer)        (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1, 30)        166980      LCLid_Input[0][0]                
__________________________________________________________________________________________________
embedding_

In [14]:
fcn_model.fit_generator(TG, epochs=10, validation_data =VG,  verbose=1,   callbacks=[
                        clr_callback.CyclicLR(max_lr=0.001, base_lr=0.00001),
                        
                        keras.callbacks.ModelCheckpoint(str(model_path/'weights.{epoch:02d}-{val_loss:.2f}.hdf5'), 
                                                              monitor='val_loss', verbose=0, 
                                                             save_best_only=True, save_weights_only=False, 
                                                               mode='auto', period=1)
                        ])

Epoch 1/10


FailedPreconditionError: Error while reading resource variable training/Adam/Variable from Container: localhost. This could mean that the variable was uninitialized. Not found: Resource localhost/training/Adam/Variable/N10tensorflow3VarE does not exist.
	 [[Node: training/Adam/mul_1/ReadVariableOp = ReadVariableOp[dtype=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](training/Adam/Variable)]]
	 [[Node: loss/mul/_61 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_902_loss/mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'training/Adam/mul_1/ReadVariableOp', defined at:
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/runp